# German traffic sign recognition 

Imagine you are working in self-driving project as a machine learning engineer. One of the important problems for self-driving cars is to follow traffic signs. A car moves and collects video from the front camera. In each separate image you could localize the signs and after that understand what sign did we find.

We can assume that someone else is responsible for localizing and tracking signs, and your task is to do the classification part.

In today's lab you will be asked to develop a traffic sign recognition system. It should take a sign image and classify which of 43 signs it belongs to.

In this problem you will be given a benchmark dataset The German Traffic Sign Recognition Benchmark: A multi-class classification competition.

Overview
1. multi-class classification problem
2. More than 40 classes
3. More than 50,000 images in total
4. Large, lifelike database
5. Reliable ground-truth data due to semi-automatic annotation
Physical traffic sign instances are unique within the dataset
(i.e., each real-world traffic sign only occurs once)

In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd
import numpy as np
import os
import torch
import torch.optim as optim
from torch.utils.data import DataLoader , Dataset
from PIL import Image
from collections import Counter
import torchvision
from torchvision import transforms
import torch.utils

In [3]:
### GET THE DATA FROM DRIVE
from google.colab import drive
drive.mount('/content/drive')
!unzip 'drive/MyDrive/GTSRB/GTSRB.zip' 

Streaming output truncated to the last 5000 lines.
 extracting: GTSRB/Train/5/00005_00053_00014.png  
 extracting: GTSRB/Train/5/00005_00053_00015.png  
 extracting: GTSRB/Train/5/00005_00053_00016.png  
 extracting: GTSRB/Train/5/00005_00053_00017.png  
 extracting: GTSRB/Train/5/00005_00053_00018.png  
 extracting: GTSRB/Train/5/00005_00053_00019.png  
 extracting: GTSRB/Train/5/00005_00053_00020.png  
 extracting: GTSRB/Train/5/00005_00053_00021.png  
 extracting: GTSRB/Train/5/00005_00053_00022.png  
 extracting: GTSRB/Train/5/00005_00053_00023.png  
 extracting: GTSRB/Train/5/00005_00053_00024.png  
 extracting: GTSRB/Train/5/00005_00053_00025.png  
 extracting: GTSRB/Train/5/00005_00053_00026.png  
 extracting: GTSRB/Train/5/00005_00053_00027.png  
 extracting: GTSRB/Train/5/00005_00053_00028.png  
  inflating: GTSRB/Train/5/00005_00053_00029.png  
 extracting: GTSRB/Train/5/00005_00054_00000.png  
 extracting: GTSRB/Train/5/00005_00054_00001.png  
 extracting: GTSRB/Train/5/0000

In [4]:
pd.read_csv("GTSRB/Train.csv")

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png
...,...,...,...,...,...,...,...,...
39204,52,56,5,6,47,51,42,Train/42/00042_00007_00025.png
39205,56,58,5,5,51,53,42,Train/42/00042_00007_00026.png
39206,58,62,5,6,53,57,42,Train/42/00042_00007_00027.png
39207,63,69,5,7,58,63,42,Train/42/00042_00007_00028.png


In [5]:
### The dataset contains Train and Test folders + Train.csv and Test.csv
### TASK : load data using csv file 

def get_data(csv_data):
    images, labels = [], []
    ### open csv file
    df = pd.read_csv("GTSRB/" + csv_data)
    ### iterate the csv , for each row get image from the 'Path' column and get label from the column 'ClassId'
    def add_entry(row):
      images.append(Image.open("GTSRB/" + row["Path"]))
      labels.append(row["ClassId"])
    
    df.apply(lambda row: add_entry(row), axis=1)
    
    return images , labels

### Next we need to build a custom Dataset in Pytorch, this dataset will be given after that to the Dataloader 

class CustomDataset(Dataset):
    def __init__(self, images, labels,transforms=None):
        self.labels = labels
        self.images = images 
        self.transform = transforms

    def __len__(self):
        ### put code here
        return len(self.labels)

    def __getitem__(self, idx):
        ### put code here
        item = {}
        item["label"] = self.labels[idx]
        item["image"] = self.transform(self.images[idx])
        # print(item)
        return item

BATCH_SIZE = 70
IMAGE_SIZE = 112
EPOCHS = 10
NB_CLASS = 43

train_transform = transforms.Compose([
    ### Define augmentations 
    ### Don't forget to convert to tensor
    transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),

    # Augmentations
    transforms.GaussianBlur(5),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                        std=[0.2023, 0.1994, 0.2010])
                                     
])

test_transform = transforms.Compose([
   transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                        std=[0.2023, 0.1994, 0.2010])
   ])



x_train,y_train = get_data('Train.csv')
train_data = CustomDataset(x_train,y_train,transforms= train_transform)
train_loader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)

x_test,y_test = get_data('Test.csv')
test_data = CustomDataset(x_test,y_test,transforms= test_transform)
test_loader = DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True)

print("Train data %.d  Test data %.d "%(len(train_loader.dataset),len(test_loader.dataset)))

Train data 39209  Test data 12630 


In [10]:
import torch.nn as nn
import torch.nn.functional as F


### If you run on GPU you must load it to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### TASK : Define 2 models the first a simple CNN and the second choose any pretrained model

class CNN_model(nn.Module):
    def __init__(self,nb_class=43):
        super(CNN_model, self).__init__()
        ### define layers here

        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 4, stride = 4)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 4, stride = 4)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, nb_class)

    def forward(self, x):
        ### call the layers here 
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

class Pretrained_model(nn.Module):
    def __init__(self,nb_class=43):
        super(Pretrained_model, self).__init__()
        ### from torchvision.models choose a pretrained model
        
        return None


model = CNN_model().to(device)

# Loss function here
loss_fn = nn.CrossEntropyLoss()
# Optimizer here
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay = 0.005, momentum = 0.9)
# Scheduler here
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.65 ** epoch)


In [7]:
import tqdm

def calculate_accuracy(y_pred, y):
    ### task : make a function that calculates the accuracy
    # Calculates the number of right predictions (not actually the accuracy)
    return sum([1 if y[i] == y_pred[i].argmax(dim=0) else 0 for i in range(len(y))])

def train(model,train_loader,loss_fn,optimizer,scheduler,device):
    
    epoch_accuracy = 0
    running_loss = 0
    total = 0
    correct = 0

    model.train()
    for i, data in tqdm.tqdm(enumerate(train_loader)):
        # task : get the images and labels and don't forget to load to device (2 lines)
        images = data["image"].to(device)
        labels = data["label"].to(device).long()

        # print(i)
        # print(data)

        # zero the parameter gradients, it's a must 
        optimizer.zero_grad()
        outputs = model(images)
        # print(outputs)
        if torch.isnan(sum(sum(outputs))) or torch.isinf(sum(sum(outputs))):
            print('invalid input detected at iteration ', i)
            break

        # task : get the loss 
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()
        
       
        running_loss += loss.item()* images.size(0)
        total += images.size(0)
        epoch_accuracy += calculate_accuracy(outputs,labels)
        
        # task : print statistics here
        # print("Loss : ", running_loss / total)
    scheduler.step()
    print("----> Training Accuracy : ", epoch_accuracy/total)
    print("----> Training Av. Loss : ", running_loss/total)
    print("New learning rate: ", optimizer.param_groups[0]["lr"])

        
def evaluate(model, loader, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    acc = 0
    total = 0
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        for i, data in tqdm.tqdm(enumerate(loader)):
            # task : get the images and labels and don't forget to load to device (2 lines)
            images = data["image"].to(device)
            labels = data["label"].to(device)

            output = model(images)
            loss = criterion(output, labels)
            
            # Calculate accuracy
            acc += calculate_accuracy(output, labels)
            total += images.size(0)
            loss += loss.item()* images.size(0)
            
            ### task : print statistics
        print("----> Evaluation Accuracy : ", acc/total)
        print("----> Evaluation Av. Loss : ", loss/total)

    

In [8]:
for epoch in range(EPOCHS):
    print("EPOCH ", epoch+1)
    train(model,train_loader,loss_fn,optimizer,scheduler,device)
    evaluate(model,test_loader,loss_fn,device)

EPOCH  1


561it [02:30,  3.73it/s]


----> Training Accuracy :  0.7030018618174398
----> Training Av. Loss :  1.089279513493613
New learning rate:  0.006500000000000001


181it [00:11, 15.48it/s]


----> Evaluation Accuracy :  0.9025336500395883
----> Evaluation Av. Loss :  tensor(0.0014, device='cuda:0')
EPOCH  2


...

----> Evaluation Accuracy :  0.9542359461599367
----> Evaluation Av. Loss :  tensor(0.0006, device='cuda:0')
EPOCH  10


561it [02:08,  4.36it/s]


----> Training Accuracy :  0.998903313014869
----> Training Av. Loss :  0.011860043357558525
New learning rate:  0.00013462743344628913


181it [00:08, 20.67it/s]

----> Evaluation Accuracy :  0.9553444180522566
----> Evaluation Av. Loss :  tensor(0.0004, device='cuda:0')


Our model turns out to be very good and retuns almost perfect accuracy

I tried using a pre-trained model but got worse accuracy so I am not including it. 